In [ ]:
#!jupyter nbconvert --to script down_pdf.ipynb

import requests
#url = 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2010-03-31/600795_2009_n.pdf'
url = 'http://disc.static.szse.cn/download/disc/disk01/finalpage/2008-04-19/cda5e777-cc3c-4934-9971-99ce9e4f6fe0.PDF'
def down_pdf(filename,url):
    success = True
    try:
        r = requests.get(url, stream=True)
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=102400):
                fd.write(chunk)
    except Exception as e:
        success = False
        print(f'download {url} faild',e)
        pass
    
    return filename,success




In [ ]:
# 检查文件合法性
import os
from PyPDF2 import PdfFileWriter, PdfFileReader
def checkPdf(f):
    """检查pdf文件的合法性"""
    # 文件是否存在
    if not os.path.exists(f):
        return False
    
    # 文件大小是否满足条件
    fsize=0
    try:
        fsize = os.path.getsize(f)
    except Exception as e:
        print(f'getsize exception {f} ',e)
        return False

    if fsize < 1024*50:
        return False
    
    # 是否正常的pdf文件
    try:
        pdfFileReader = PdfFileReader(f)
        if pdfFileReader.isEncrypted:
            pdfFileReader.decrypt('')
        if pdfFileReader.getDocumentInfo()  is not None:
            return True
    except:
        pass
    return False
    

In [ ]:
import time
import json
from tqdm import tqdm

# python  down_pdf.py 3 1 表示启动三个进程，除以3取摸为1的进行
import sys
a1,a2=1,1
print("sys.argv=",sys.argv)
if len(sys.argv)==3:
    a1=int(sys.argv[1])
    a2=int(sys.argv[2])
    
def down_loop():
    with open('pdf_urls.json') as json_file:  
        pdf_urls = json.load(json_file)

    keys = list(pdf_urls.keys())
    base_dir='../scraping/content/'
    pdf_dir ='../scraping/pdf/'
    for fileName in tqdm(keys):
        code = int(fileName[:6])
        url = pdf_urls[fileName]
        if code % a1 !=a2:
#             print('jump',code)
            continue

        pdf_file=pdf_dir+fileName+'.pdf'
        if checkPdf(pdf_file):
            print(f'{pdf_file} is ok,jump...')
            continue

        times = 0
        while times<= 3:
            times +=1
            t0=time.time()
            print(f'开始下载文件：{url},times={times}')
            pdf_file,success = down_pdf(pdf_file,url)
            if not success:
                time.sleep(2)
                continue
            fsize=1
            try:
                fsize = os.path.getsize(pdf_file)
            except:
                pass
            print(f'下载文件({pdf_file})成功，用时(秒):{time.time()-t0},times={times},fsize={fsize/1024}KB')
            t0=time.time()
            if checkPdf(pdf_file):
                print('+'*30,f'校验文件({pdf_file})成功')
                break
            else:
                print('-'*30,f'校验文件({pdf_file})失败，times={times}')
    
    time.sleep(120)
    
    
down_loop()
    